In [40]:
import torch
import torch.nn as nn

## save model with path

In [41]:
from pathlib import Path

#* 1 - create model idrecotry 
model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

#* 2- create model save path 
model_name = "model_0.pth"
model_save_path = model_path/model_name

model_save_path

WindowsPath('models/model_0.pth')

In [42]:
#* creating the model 

class liner_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(
            torch.randn(1, dtype=torch.float32, requires_grad=True)
        )
        self.bias = nn.Parameter(
            torch.randn(1, dtype=torch.float32, requires_grad=True)
        )
    
    def forward(self, x):
        return self.weights * x + self.bias

    

In [43]:
model = liner_model()
model.load_state_dict(torch.load(model_save_path)) #* takes a path to the model

#* saving
#torch.save(obj=model.state_dict(),f = model_save_path)

<All keys matched successfully>

In [44]:
#* create torchScript

model_script = torch.jit.script(model)

model_script.save("model_script.pt")
model = torch.jit.load("model_script.pt")

model.eval()

RecursiveScriptModule(original_name=liner_model)

## Saving and loading a general checkpoint for inference or resuming training

In [45]:
model = liner_model()
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(params=model.parameters(), lr = 0.01)
epoch = 500

In [46]:
weights = 0.8
bias = 0.2
start = 0
end= 2
step = 0.04
X = torch.arange(start, end, step)
y = weights*X + bias
train_size = int(0.8*len(X))
xtrain = X[:train_size]
ytrain = y[:train_size]
xtest = X[train_size:]
ytest = y[train_size:]


In [47]:
for epoch in range(epoch):
    model.train()
    y_pred = model(xtrain)
    loss = loss_fn(y_pred, ytrain)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    model.eval()
    with torch.inference_mode():
        y_test = model(xtest)
        tes_loss = loss_fn(y_test, ytest)
        if epoch % 50 == 0:
            print(f"epoch: {epoch}, loss: {loss.item():.4f}, test_loss: {tes_loss.item():.4f}")
        
        
    

epoch: 0, loss: 0.9224, test_loss: 2.5716
epoch: 50, loss: 0.7853, test_loss: 2.0572
epoch: 100, loss: 0.7009, test_loss: 1.6897
epoch: 150, loss: 0.6411, test_loss: 1.4317
epoch: 200, loss: 0.5913, test_loss: 1.2551
epoch: 250, loss: 0.5443, test_loss: 1.1285
epoch: 300, loss: 0.4983, test_loss: 1.0256
epoch: 350, loss: 0.4523, test_loss: 0.9226
epoch: 400, loss: 0.4064, test_loss: 0.8197
epoch: 450, loss: 0.3604, test_loss: 0.7168


In [48]:
model(xtest)

tensor([0.9989, 1.0010, 1.0030, 1.0050, 1.0070, 1.0091, 1.0111, 1.0131, 1.0152,
        1.0172], grad_fn=<AddBackward0>)

In [49]:
torch.save(
    {
        'epoch': epoch,
        'model_state_dict' : model.state_dict(),
        'optimizer_state_dict' : optimizer.state_dict(),
        'loss' : loss,
    },
    "training_pt_1.pt"
)

In [50]:
model = liner_model()
optimizer = torch.optim.SGD(params=model.parameters(), lr = 0.01)
loss_fn = nn.L1Loss()

In [51]:
checkpoint = torch.load("training_pt_1.pt")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
model.eval()


liner_model()

In [52]:
model(xtest)

tensor([0.9989, 1.0010, 1.0030, 1.0050, 1.0070, 1.0091, 1.0111, 1.0131, 1.0152,
        1.0172], grad_fn=<AddBackward0>)

In [53]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

## moving model to gpu

1. load model 
2. move to gpu
3. save model 
4. reload model

In [68]:
torch.save(model.state_dict(), "model_gpu.pth")


In [64]:
model.to('cuda')
next(model.parameters()).is_cuda



True

In [65]:
model = liner_model()

In [73]:
model.load_state_dict(torch.load("model_gpu.pth",  map_location=torch.device('cuda')))
model.to('cuda')
next(model.parameters()).is_cuda

True

'NVIDIA GeForce GTX 1650'